In [9]:
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer
from fastlane_bot.tools.univ3calc import Univ3Calculator as U3
from dataclasses import dataclass, asdict

print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(U3))

from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

ConstantProductCurve v2.6.4.2 (30/Apr/2023)
Univ3Calculator v1.1 (19/Apr/2023)
Version = 2.0-BETA4 [requirements >= 2.0 is met]


# Uniswap code [NBTest066]

## u3 standalone

In [10]:
data = {
    "token0": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", # USDC
    "token1": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", # WETH 
    "sqrt_price_q96": "1725337071198080486317035748446190", 
    "tick": "199782", 
    "liquidity": "36361853546581410773"
}

In [11]:
u1 = U3(
    tkn0="USDC", 
    tkn0decv=6, 
    tkn1="WETH", 
    tkn1decv=18,
    sp96=data["sqrt_price_q96"],
    tick=data["tick"],
    liquidity=data["liquidity"],
    fee_const = U3.FEE500,
)
u2 = U3.from_dict(data, U3.FEE500)
assert u1 == u2
u = u2
assert asdict(u) == {
    'tkn0': 'USDC',
    'tkn1': 'WETH',
    'sp96': int(data["sqrt_price_q96"]),
    'tick': int(data["tick"]),
    'liquidity': int(data["liquidity"]),
    'fee_const': U3.FEE500
}
assert u.tkn0 == "USDC"
assert u.tkn1 == "WETH"
assert u.tkn0dec == 6
assert u.tkn1dec == 18
assert u.decf == 1e-12
assert u.dec_factor_wei0_per_wei1 == u.decf
assert iseq(u.p, 0.00047422968986928404)
assert iseq(1/u.p, 2108.6828205033694)
assert u.p == u.price_tkn1_per_tkn0
assert 1/u.p == u.price_tkn0_per_tkn1
assert u.price_convention == 'USDC/WETH [WETH per USDC]'
assert iseq(u._price_f(1725337071198080486317035748446190), 474229689.86928403)
assert iseq(u._price_f(u.sp96), 474229689.86928403)
assert u.ticksize == 10
ta, tb =  u.tickab
par, pbr = u.papb_raw
pa, pb = u.papb_tkn1_per_tkn0
pai, pbi = u.papb_tkn0_per_tkn1
assert ta <= u.tick
assert tb >= u.tick
assert ta % u.ticksize == 0
assert tb % u.ticksize == 0
assert tb-ta == u.ticksize
assert iseq(par, 474134297.0246954)
assert iseq(pbr, 474608644.73905975)
assert iseq(pbr/par, 1.0001**u.ticksize)
assert iseq(pa, 0.0004741342970246954)
assert iseq(pb, 0.00047460864473905973)
assert iseq(pbr/par, pb/pa)
assert iseq(pbr/par, pai/pbi)
assert pa<pb
assert pai>pbi
assert pa == par * u.decf
assert pb == pbr * u.decf
assert iseq(pai, 2109.1070742514007)
assert iseq(pbi, 2106.999126722188)
assert pai == 1/pa
assert pbi == 1/pb
assert u.p >= pa
assert u.p <= pb
assert u.fee_const == 500
assert u.fee == 0.0005
assert u.info()
print(u.info())

Uniswap v3 Range USDC/WETH (fee=0.05%)
  Pa =        0.000   P=       0.000   Pb =        0.000 WETH per USDC
1/Pa =    2,109.107 1/P=   2,108.683 1/Pb =    2,106.999 USDC per WETH
---
 full P = 0.00047422968986928404, full 1/P = 2108.6828205033694



In [12]:
assert u.liquidity == int(data["liquidity"])
assert u.L == 36361853.54658141
assert u.liquidity/u.L == 1e18/1e6
assert u.L2 == u.L**2
assert u.Lsquared == u.L**2
assert u.k == u.L2
assert u.kbar == u.L
u.tkn0reserve(incltoken=True), u.tkn1reserve(incltoken=True), u.tvl(incltoken=True)

((666745.6837363543, 'USDC'),
 (79.64514158623042, 'WETH'),
 (395.83574040620545, 'WETH'))

## with cpc

In [13]:
data = {
    "token0": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48", 
    "token1": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", 
    "sqrt_price_q96": "1727031172247131125466697684053376", 
    "tick": "199801", 
    "liquidity": "37398889145617323159"
}
u = U3.from_dict(data, U3.FEE500)

In [14]:
pa, pb = u.papb_tkn1_per_tkn0
curve = CPC.from_univ3(
    Pmarg = u.p,
    uniL = u.L,
    uniPa = pa,
    uniPb = pb,
    pair = u.pair,
    fee = u.fee,
    descr = "",
    params = dict(uv3raw=asdict(u)),
    cid = "0",
)
curve

ConstantProductCurve(k=1398676909326173.8, x=1715687194.7056706, x_act=716877.5715601444, y_act=66.88731140131131, pair='USDC/WETH', cid='0', fee=0.0005, descr='', constr='pkpp', params={'uv3raw': {'tkn0': 'USDC', 'tkn1': 'WETH', 'sp96': 1727031172247131125466697684053376, 'tick': 199801, 'liquidity': 37398889145617323159, 'fee_const': 500}, 'L': 37398889.14561733})

In [15]:
c = curve
print(f"Reserve: {c.x_act:,.3f} {c.tknx}, {c.y_act:,.3f} {c.tkny}")
print(f"TVL = {c.tvl(tkn=c.tknx):,.3f} {c.tknx} = {c.tvl(tkn=c.tkny):,.3f} {c.tkny}")
assert iseq(c.x_act, 716877.5715601444)
assert iseq(c.y_act, 66.88731140131131)
assert iseq(c.tvl(tkn=c.tknx), 857645.1222000704)
assert iseq(c.tvl(tkn=c.tkny), 407.51988721569177)

Reserve: 716,877.572 USDC, 66.887 WETH
TVL = 857,645.122 USDC = 407.520 WETH


In [16]:
print(f"Reserve: {u.tkn0reserve():,.3f} {c.tknx}, {u.tkn1reserve():,.3f} {c.tkny}")
print(f"TVL = {u.tvl(astkn0=True):,.3f} {c.tknx} = {u.tvl(astkn0=False):,.3f} {c.tkny}")
assert iseq(u.tkn0reserve(), c.x_act)
assert iseq(u.tkn1reserve(), c.y_act)
assert iseq(u.tvl(astkn0=False), c.tvl(tkn=c.tkny))
assert iseq(u.tvl(astkn0=True), c.tvl(tkn=c.tknx))
assert u.tkn0reserve(incltoken=True)[1] == u.tkn0
assert u.tkn1reserve(incltoken=True)[1] == u.tkn1
assert u.tvl(astkn0=True, incltoken=True)[1] == u.tkn0
assert u.tvl(astkn0=False, incltoken=True)[1] == u.tkn1
u.tkn0reserve(incltoken=True), u.tkn1reserve(incltoken=True), u.tvl(incltoken=True)

Reserve: 716,877.572 USDC, 66.887 WETH
TVL = 857,645.122 USDC = 407.520 WETH


((716877.571560313, 'USDC'),
 (66.8873114012806, 'WETH'),
 (407.5198872157411, 'WETH'))

In [17]:
curve = CPC.from_univ3(
    **u.cpc_params(),
    descr = "",
    params = dict(uv3raw=asdict(u)),
    cid = "0",
)
curve

ConstantProductCurve(k=1398676909326173.8, x=1715687194.7056706, x_act=716877.5715601444, y_act=66.88731140131131, pair='USDC/WETH', cid='0', fee=0.0005, descr='', constr='pkpp', params={'uv3raw': {'tkn0': 'USDC', 'tkn1': 'WETH', 'sp96': 1727031172247131125466697684053376, 'tick': 199801, 'liquidity': 37398889145617323159, 'fee_const': 500}, 'L': 37398889.14561733})

In [18]:
c = curve
print(f"Reserve: {c.x_act:,.3f} {c.tknx}, {c.y_act:,.3f} {c.tkny}")
print(f"TVL = {c.tvl(tkn=c.tknx):,.3f} {c.tknx} = {c.tvl(tkn=c.tkny):,.3f} {c.tkny}")

Reserve: 716,877.572 USDC, 66.887 WETH
TVL = 857,645.122 USDC = 407.520 WETH


In [19]:
curve = CPC.from_univ3(
    **u.cpc_params(
        cid = "0",
        descr = "",
        #params = dict(uv3raw=asdict(u)),
    ),
)
curve

ConstantProductCurve(k=1398676909326173.8, x=1715687194.7056706, x_act=716877.5715601444, y_act=66.88731140131131, pair='USDC/WETH', cid='0', fee=0.0005, descr='', constr='pkpp', params={'L': 37398889.14561733})

In [20]:
c = curve
print(f"Reserve: {c.x_act:,.3f} {c.tknx}, {c.y_act:,.3f} {c.tkny}")
print(f"TVL = {c.tvl(tkn=c.tknx):,.3f} {c.tknx} = {c.tvl(tkn=c.tkny):,.3f} {c.tkny}")

Reserve: 716,877.572 USDC, 66.887 WETH
TVL = 857,645.122 USDC = 407.520 WETH
